# Exercise 02 : Join

In [48]:
import pandas as pd
import sqlite3

### create a connection to the database using the library sqlite3

In [49]:
con = sqlite3.connect('../data/checking-logs.sqlite')
con.execute("DROP TABLE IF EXISTS datamart;")

### create a new table datamart in the database by joining the tables pageviews and checker using only one query

* the table should have the following columns: uid, labname, first_commit_ts, first_view_ts
* first_commit_ts is just a new name of the column timestamp from the checker table, it shows the first commit from a particular lab and from a particular user
* first_view_ts is the first visit of a user to the table pageviews, timestamp when a user visited the newsfeed
* status = ’ready’ should still be a filter
* numTrials = 1 should still be a filter
* labnames should still be from the list: ’laba04’, ’laba04s’, ’laba05’, ’laba06’, ’laba06s’, ’project1’
* the table should contain only the users (uids with user_*) and not the admins
* first_commit_ts and first_view_ts should be parsed as datetime64[ns]




In [50]:
con.execute("""CREATE TABLE datamart AS
SELECT c.uid,
       c.labname,
       MIN(c.timestamp) AS first_commit_ts,
       MIN(p.datetime)  AS first_view_ts
FROM checker c
         LEFT JOIN
     pageviews p ON c.uid = p.uid
WHERE c.status = 'ready'
  AND c.numTrials = 1
  AND c.labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
  AND c.uid LIKE 'user_%'
GROUP BY c.uid, c.labname;
            """)

### using Pandas methods, create two dataframes: test and control

* test should have the users that have the values in first_view_ts
* control should have the users that have missing values in first_view_ts
* replace the missing values in the control with the average first_view_ts of the test users, we will use this value for the future analysis
* save both tables into the database, you will use them in the next exercises

In [51]:
datamart = pd.read_sql("SELECT * FROM datamart", con)

In [52]:
datamart = datamart.astype({'first_commit_ts': 'datetime64[ns]', 'first_view_ts': 'datetime64[ns]'})

In [53]:
test = datamart[datamart['first_view_ts'].notna()].copy()
control = datamart[datamart['first_view_ts'].isna()].copy()

In [54]:
control['first_view_ts'] = test['first_view_ts'].mean()

In [55]:
control

,uid,labname,first_commit_ts,first_view_ts
12,user_11,laba05,2020-05-03 21:06:55.970293,2020-04-27 00:40:05.761783552
13,user_11,project1,2020-05-03 23:45:33.673409,2020-04-27 00:40:05.761783552
14,user_12,laba04,2020-04-18 17:07:51.767358,2020-04-27 00:40:05.761783552
15,user_12,laba04s,2020-04-26 15:42:38.070593,2020-04-27 00:40:05.761783552
16,user_12,laba05,2020-05-03 08:39:25.174316,2020-04-27 00:40:05.761783552
...,...,...,...,...
135,user_8,laba04s,2020-04-19 10:22:35.761944,2020-04-27 00:40:05.761783552
136,user_8,laba05,2020-05-02 13:28:07.705193,2020-04-27 00:40:05.761783552
137,user_8,laba06,2020-05-16 17:56:15.755553,2020-04-27 00:40:05.761783552
138,user_8,laba06s,2020-05-16 20:01:07.900727,2020-04-27 00:40:05.761783552


In [56]:
test.to_sql("test", con, if_exists="replace", index=False)
control.to_sql("control", con, if_exists="replace", index=False)

81

### close the connection

In [57]:
con.close()